In [25]:
import sys

# sys.path.append("../../src/stong_core")
sys.path.append("../../src/stong_core_cython")

In [26]:
import numpy as np
import gaussian_overlap
import importlib
import h5py
import pandas as pd
from scipy.linalg import eigh
import matplotlib.pyplot as plt
import copy
import hf_sto_3g_diis
import sto_ng_matrix

from scipy.optimize import minimize

importlib.reload(gaussian_overlap)
importlib.reload(hf_sto_3g_diis)
importlib.reload(sto_ng_matrix)
from gaussian_overlap import *
from sto_ng_matrix import *
from hf_sto_3g_diis import *

In [27]:
df = pd.read_csv("../make_base/sto_3g.csv")
df

,exps_1,coefs_1s,exps_2,coefs_2s,coefs_2p
0,0.109987,0.445576,0.074759,0.694673,0.387976
1,0.407154,0.535362,0.229124,0.405616,0.608867
2,2.246014,0.153482,0.979606,-0.100240,0.158557


In [69]:
zeta_1s = 2.69
zeta_2sp = 0.75
Z = 2
n_up = Z // 2 + Z % 2
n_dn = Z // 2

scf(zeta_1s, zeta_2sp, df, n_up, n_dn, Z)

[[ 4.44089210e-16 -1.31627842e+00 -1.77889534e+00 -1.77889534e+00
  -1.77889534e+00]
 [ 1.31627842e+00  0.00000000e+00 -6.99082970e-01 -6.99082970e-01
  -6.99082970e-01]
 [ 1.77889534e+00  6.99082970e-01  0.00000000e+00  0.00000000e+00
  -5.55111512e-17]
 [ 1.77889534e+00  6.99082970e-01  0.00000000e+00  0.00000000e+00
  -5.55111512e-17]
 [ 1.77889534e+00  6.99082970e-01  5.55111512e-17  5.55111512e-17
   0.00000000e+00]]
[[ 4.44089210e-16 -1.31627842e+00 -1.77889534e+00 -1.77889534e+00
  -1.77889534e+00]
 [ 1.31627842e+00  0.00000000e+00 -6.99082970e-01 -6.99082970e-01
  -6.99082970e-01]
 [ 1.77889534e+00  6.99082970e-01  0.00000000e+00  0.00000000e+00
  -5.55111512e-17]
 [ 1.77889534e+00  6.99082970e-01  0.00000000e+00  0.00000000e+00
  -5.55111512e-17]
 [ 1.77889534e+00  6.99082970e-01  5.55111512e-17  5.55111512e-17
   0.00000000e+00]]
[[ 0.          0.00318476  0.          0.          0.        ]
 [-0.00318476  0.          0.          0.          0.        ]
 [ 0.          0.     

{'energy': -2.324153117545739,
 'c_up': array([[-0.75207573, -0.50896107,  0.        ,  0.        ,  0.        ]]),
 'c_dn': array([[-0.75207573, -0.50896107,  0.        ,  0.        ,  0.        ]]),
 'D_up': array([[0.560752  , 0.38846371, 0.        , 0.        , 0.        ],
        [0.38846371, 0.26911015, 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ]]),
 'D_dn': array([[0.560752  , 0.38846371, 0.        , 0.        , 0.        ],
        [0.38846371, 0.26911015, 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ]])}

In [65]:
c_up = np.zeros((n_up, 5))
c_dn = np.zeros((n_dn, 5))
Tmat, Vmat, ERImat, Smat = make_matrix(zeta_1s, zeta_2sp, df, Z)
hmat = Tmat + Vmat
new_energy = 0
D_up = np.ones((5, 5))
D_dn = np.ones((5, 5))

In [66]:
Smat

array([[1.        , 0.21898809, 0.        , 0.        , 0.        ],
       [0.21898809, 1.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 1.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 1.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 1.        ]])

In [68]:
# Fock行列の構築
Fmat_up_old, Jmat_up, Kmat_up = build_Fock_matrix_up(
    ERImat, hmat, c_up, c_dn, n_up, n_dn
)
Fmat_dn_old, Jmat_dn, Kmat_dn = build_Fock_matrix_dn(
    ERImat, hmat, c_up, c_dn, n_up, n_dn
)

# 対角化
val_up, vec_up = eigh(Fmat_up_old)
val_dn, vec_dn = eigh(Fmat_dn_old)
phi0 = vec_up[:, 0]
phi1 = vec_up[:, 1]
phi2 = vec_up[:, 2]
phi3 = vec_up[:, 3]


# 密度行列の構築
P0 = np.zeros((5, 5))
P1 = np.zeros((5, 5))
P2 = np.zeros((5, 5))
P3 = np.zeros((5, 5))

for i in range(5):
    for j in range(5):
        P0[i, j] = phi0[i] * phi0[j]
        P1[i, j] = phi1[i] * phi1[j]
        P2[i, j] = phi2[i] * phi2[j]
        P3[i, j] = phi3[i] * phi3[j]

#  残差
err_up = (
    Smat @ (P0 + P1 + P2 + P3) @ Fmat_up_old - Fmat_up_old @ (P0 + P1 + P2 + P3) @ Smat
)

In [31]:
D0 = np.array([vec_up[:, 0]]).T @ np.array([vec_up[:, 0]])
D1 = np.array([vec_up[:, 1]]).T @ np.array([vec_up[:, 1]])

In [32]:
c_up = vec_up[:, :n_up].T
c_dn = vec_dn[:, :n_dn].T

# Fock行列の構築
Fmat_up_new, Jmat_up, Kmat_up = build_Fock_matrix_up(
    ERImat, hmat, c_up, c_dn, n_up, n_dn
)
Fmat_dn_new, Jmat_dn, Kmat_dn = build_Fock_matrix_dn(
    ERImat, hmat, c_up, c_dn, n_up, n_dn
)

In [33]:
Fmat_up_old @ D0 @ Smat - Smat @ D0 @ Fmat_up_old

array([[1.77635684e-15, 8.88178420e-16, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [0.00000000e+00, 5.55111512e-17, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00]])

In [34]:
Fmat_up_old @ D1 @ Smat - Smat @ D1 @ Fmat_up_old

array([[-6.93889390e-18,  8.32667268e-17,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [-2.77555756e-16, -4.44089210e-16,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00]])

In [35]:
Fmat_up_new @ D0 @ Smat - Smat @ D0 @ Fmat_up_new

array([[ 1.77635684e-15, -2.28979679e-02,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 2.28979679e-02,  2.77555756e-17,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00]])

In [36]:
Fmat_up_new @ D1 @ Smat - Smat @ D1 @ Fmat_up_new

array([[ 3.20923843e-17,  2.28979679e-02,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [-2.28979679e-02, -1.11022302e-16,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00]])